## Classifying news-wires a multiclass classification:
This problem is a instance of multiclass classification
because each data point should be classified into one category ,
the problem is more specifically an instance of single label multiclass
classification.

### Reuters dataset


In [16]:
from tensorflow.keras.datasets import reuters
(train_data,train_labels) , (test_data, test_labels) = reuters.load_data(
    num_words=10000
)

print(len(train_data))
print(len(test_data))

8982
2246


/home/mgrdich/.local/share/virtualenvs/deep_learning_python-pzRhtxR0/lib/python3.8/site-packages/tensorflow/python/keras/datasets/reuters.py:143: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/home/mgrdich/.local/share/virtualenvs/deep_learning_python-pzRhtxR0/lib/python3.8/site-packages/tensorflow/python/keras/datasets/reuters.py:144: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


### Decoding news-wires back to text



In [17]:
word_index = reuters.get_word_index()
reverse_word_index = {v: k for k, v in word_index.items()}
decoded_newswire = ''.join([
    reverse_word_index.get(i-3,'?') for i in train_data[0]
])
print(decoded_newswire)

???saidasaresultofitsdecemberacquisitionofspacecoitexpectsearningspersharein1987of115to130dlrspershareupfrom70ctsin1986thecompanysaidpretaxnetshouldrisetonineto10mlndlrsfromsixmlndlrsin1986andrentaloperationrevenuesto19to22mlndlrsfrom125mlndlrsitsaidcashflowpersharethisyearshouldbe250tothreedlrsreuter3


the label associated with an example is an integer between `0-45` topic index

In [18]:
train_labels[0]
print(reverse_word_index[train_labels[0]])
# probably not decoded this way


to


In [19]:
# vectorized
from util.functions import vectorize_sequences
x_train = vectorize_sequences(train_data,10000)
x_test = vectorize_sequences(test_data,10000)

# OR categorical encoding
from tensorflow.keras.utils import to_categorical
one_hot_train_labels = to_categorical(train_labels)
one_hot_test_labels = to_categorical(test_labels)


### Building your Network
But there is a new constraint here: the number of output classes has gone from 2 to 46. The
dimensionality of the output space is much larger.

In Stack of `Dense` Layers like you've been using each layer can only access information
present in the output the previous layer

In [ ]:
from tensorflow.keras import models , layers
model = models.Sequential()
model.add(layers.Dense(64,activation='relu', input_shape=(10000,)))
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(46,activation='softmax'))

There are two things two know about this architecture
* You end the network with a dense layer of size 46. Each entry in this vec-
tor (each dimension) will encode a different output class.
* The last layer uses `softmax` activation , the network will output
`probability Distribution` over `46` different output classes
`output[i]` is the probability that the sample belongs to class `i`. The scores
`46` will sum to `1`.

`categorical_crossentropy` . It measures the distance between two probability distributions

In [ ]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy']
              )